In [ ]:
from bisect import bisect_right
import random
from typing import List, Tuple

def make_histogram(x: List[float], nbins: int) -> Tuple[List[float], List[int]]:
    """
    Return (bin_edges, counts) using equal-width bins on [min(x), max(x)].
    Rightmost bin is closed; others are half-open: [e[i], e[i+1]).
    """
    if nbins < 1 or not x:
        raise ValueError("nbins>=1 and x non-empty")
    lo, hi = min(x), max(x)
    if lo == hi:
        # all points identical: put everything into the last bin
        edges = [lo + (hi - lo) * i / nbins for i in range(nbins + 1)]
        counts = [0]*(nbins-1) + [len(x)]
        return edges, counts

    width = (hi - lo) / nbins
    edges = [lo + i * width for i in range(nbins + 1)]
    counts = [0] * nbins
    for v in x:
        j = min(int((v - lo) / width), nbins - 1)  # put max into last bin
        counts[j] += 1
    return edges, counts

def update_histogram(edges: List[float], counts: List[int], new_x: List[float]) -> None:
    """
    In-place update of counts for a batch of new values. Values outside range
    are clipped into the end bins.
    """
    nbins = len(counts)
    lo, hi = edges[0], edges[-1]
    for v in new_x:
        if v <= lo: 
            counts[0] += 1
            continue
        if v >= hi:
            counts[-1] += 1
            continue
        # find i with edges[i] <= v < edges[i+1]
        i = bisect_right(edges, v) - 1
        i = min(max(i, 0), nbins - 1)
        counts[i] += 1

def update_histogram_no_bisect(edges, counts, new_x):
    nbins = len(counts)
    lo, hi = edges[0], edges[-1]
    width = (hi - lo) / nbins  # equal-width bins

    for v in new_x:
        if v <= lo:
            i = 0
        elif v >= hi:
            i = nbins - 1
        else:
            # map v into [0, nbins) and floor
            i = int((v - lo) / width)
            # numerical safety: if v is extremely close to hi, clamp
            if i >= nbins: 
                i = nbins - 1
        counts[i] += 1
        

def histogram_pdf(edges: List[float], counts: List[int]) -> List[float]:
    """Return per-bin probabilities (sum to 1)."""
    n = sum(counts)
    if n == 0:
        return [0.0]*len(counts)
    return [c / n for c in counts]

def sample_from_histogram(edges: List[float], counts: List[int]) -> float:
    """
    Draw one sample:
    1) pick a bin with prob proportional to its count,
    2) then sample uniformly within that bin.
    """
    n = sum(counts)
    if n == 0:
        raise ValueError("empty histogram")
    # choose bin index
    r = random.randrange(n)  # 0..n-1
    cum = 0
    for i, c in enumerate(counts):
        cum += c
        if r < cum:
            # uniform in bin [edges[i], edges[i+1]) except last bin closed
            a, b = edges[i], edges[i+1]
            # handle degenerate zero-width bin
            return a if b == a else (a + (b - a) * random.random())
    # fallback (shouldn’t reach here)
    return edges[-2]

def sample_many(edges: List[float], counts: List[int], m: int) -> List[float]:
    return [sample_from_histogram(edges, counts) for _ in range(m)]

In [2]:
data = [1.2, 2.7, 3.1, 2.0, 5.0, 1.9, 2.2, 2.2, 3.8, 4.1]
edges, counts = make_histogram(data, nbins=4)
print("edges:", edges)
print("counts:", counts)

# Update with new observations
new_batch = [2.5, 2.6, 4.8]
update_histogram(edges, counts, new_batch)
print("updated counts:", counts)

# Probabilities per bin (histogram as a discrete distribution)
probs = histogram_pdf(edges, counts)
print("bin probs:", probs)

# Simulate from the histogram
samples = sample_many(edges, counts, m=5)
print("simulated:", samples)

edges: [1.2, 2.15, 3.0999999999999996, 4.05, 5.0]
counts: [3, 3, 2, 2]
updated counts: [3, 5, 2, 3]
bin probs: [0.23076923076923078, 0.38461538461538464, 0.15384615384615385, 0.23076923076923078]
simulated: [3.823669582262859, 2.83070939273921, 2.263637316423909, 4.4531956871567, 2.3433215897738586]
